# CNN from Sctrach for Stanford Dog Dataset

## Importing liberaries

In [1]:
import tensorflow as tf
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.layers import MaxPool2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
import os,sys
%matplotlib inline

1. Loading Data using Keras ImageDatagenerator <br/>
Params : <br/>
rescale all images and divide by 255 for color<br/>
Shear mapping or shear range set to 0.2<br/>
zoom image with 20% or 0.2<br/>
horizontal flip image set to true<br/>

above params to create image generator for training<br/>

2. Using flow_from_directory to create image dataset from created ImageDatagenerator<br/>
Params : <br/>
a. training folder (annotated image after applying crop according to the annotation from dataset ref. Prepping_Data)<br/>
b. target image size (I have used (128X128), can be chhanged to 256X256 or even smaller (64X64), check with what size image details is not diminished )<br/>
c. batch size = 32<br/>
d. class_mode = 'categorical' if output is of more than two output in case of two output - 'binary'

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'anotated/train',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

Found 12000 images belonging to 120 classes.


Same as training Data but some changes as we are not doing any transformation on test dataset<br/>
1. For ImageDataGenerator only use rescale of colors<br/>

2. For Dataset generation use same as training data <br/>

In [3]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'anotated/test',
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

Found 8580 images belonging to 120 classes.


Initilizing CNN

In [4]:
cnn = Sequential()

Add first CNN layer <br/>
Using 3X3 Convolution activation relu and input shape is of 128X128X3 (for rgb) <br/>
Using MaxPool2d<br/>
Introdcing noise/regularization using dropout layer with 20% params drop<br/>

In [5]:
cnn.add(Conv2D(filters=32, kernel_size=3, activation='relu',input_shape=[128, 128, 3]))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))

Adding Second Layer of CNN

In [6]:
cnn.add( Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))

Adding Third Layer of CNN

In [7]:
cnn.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))

Adding Fourth Layer of CNN

In [8]:
cnn.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))

Adding GlobalAveragePooling on final params

In [9]:
cnn.add(GlobalAveragePooling2D())

Adding Dense layer with activation function of 'softmax'

In [10]:
cnn.add(Dense(units=120, activation='softmax'))

Final Summary of CNN layer <br/>
Total params: 419,256<br/>
Trainable params: 419,256<br/>

In [11]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 128)       7

## Compiling CNN
Optimizer Used : Stochastic Gradient Descent (Adam in tf)<br/>
loss function : for categorical feature training use 'categorical_crossentropy' <br/>
metrics : 'accuracy'

In [12]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

Fitting Data in Compiled CNN <br/>
validation data = test_set <br/>
epochs = 25

In [13]:
with tf.device("/device:GPU:0"):
    cnn.fit(training_set, validation_data = test_set, epochs = 25 )

Epoch 1/25
375/375 [==============================] - 790s 2s/step - loss: 4.6757 - accuracy: 0.0133 - val_loss: 4.4649 - val_accuracy: 0.0291
Epoch 2/25
375/375 [==============================] - 297s 793ms/step - loss: 4.4319 - accuracy: 0.0268 - val_loss: 4.3938 - val_accuracy: 0.0364
Epoch 3/25
375/375 [==============================] - 98s 261ms/step - loss: 4.3293 - accuracy: 0.0391 - val_loss: 4.2204 - val_accuracy: 0.0540
Epoch 4/25
375/375 [==============================] - 99s 264ms/step - loss: 4.1579 - accuracy: 0.0584 - val_loss: 3.9971 - val_accuracy: 0.0774
Epoch 5/25
375/375 [==============================] - 98s 261ms/step - loss: 3.9904 - accuracy: 0.0796 - val_loss: 3.9222 - val_accuracy: 0.0955
Epoch 6/25
375/375 [==============================] - 99s 264ms/step - loss: 3.8518 - accuracy: 0.1001 - val_loss: 3.8732 - val_accuracy: 0.0967
Epoch 7/25
375/375 [==============================] - 102s 273ms/step - loss: 3.7334 - accuracy: 0.1129 - val_loss: 3.6769 - val_ac

Fitting Data in Compiled CNN <br/>
validation data = test_set <br/>
epochs = 35

In [14]:
with tf.device("/device:GPU:0"):
    cnn.fit(training_set, validation_data = test_set, epochs = 35 )

Epoch 1/35
375/375 [==============================] - 72s 193ms/step - loss: 2.4905 - accuracy: 0.3542 - val_loss: 2.9663 - val_accuracy: 0.2740
Epoch 2/35
375/375 [==============================] - 82s 218ms/step - loss: 2.4414 - accuracy: 0.3619 - val_loss: 2.9988 - val_accuracy: 0.2650
Epoch 3/35
375/375 [==============================] - 96s 255ms/step - loss: 2.4029 - accuracy: 0.3730 - val_loss: 3.0119 - val_accuracy: 0.2608
Epoch 4/35
375/375 [==============================] - 93s 247ms/step - loss: 2.3668 - accuracy: 0.3750 - val_loss: 2.9046 - val_accuracy: 0.2822
Epoch 5/35
375/375 [==============================] - 92s 245ms/step - loss: 2.3466 - accuracy: 0.3837 - val_loss: 3.0328 - val_accuracy: 0.2723
Epoch 6/35
375/375 [==============================] - 494s 1s/step - loss: 2.2923 - accuracy: 0.3997 - val_loss: 2.9855 - val_accuracy: 0.2767
Epoch 7/35
375/375 [==============================] - 154s 410ms/step - loss: 2.2871 - accuracy: 0.3930 - val_loss: 2.9043 - val_acc

CNN are trained on training Data and Validated on test data<br/>
<br/>
for epochs : 25 <br/>
loss: 2.5072 - accuracy: 0.3517 - val_loss: 3.0804 - val_accuracy: 0.2503 <br/>
<br/>
<br/>
for epochs : 35 <br/>
loss: 1.7838 - accuracy: 0.5076 - val_loss: 2.8887 - val_accuracy: 0.3207